<a href="https://colab.research.google.com/github/ShenggKai/Kaggle_TempoRun/blob/master/Notebook/Meowmeow_TempoRun_InformationRetrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>
<img src="https://i.imgur.com/MZQ1o3n.png" alt="winer"/>
</center>

[[AI TEMPO RUN] Retrieval of Song Lyrics](https://www.kaggle.com/c/temporun-retrieval-of-song-lyrics/)

# My team

* ✈ Name: Meowmeow  
* 🕶 Team members: Nguyễn Quốc Đạt - [ShengKaii](https://www.kaggle.com/shengkaii)

# Setup

## Install and import required library

In [1]:
import os
import sys
import re
import pprint as pp
import numpy as np
import pandas as pd
import json
import csv
# import tensorflow as tf 

# from official.modeling import tf_utils
# from official import nlp
# from official.nlp import bert

# # Load the required submodules
# import official.nlp.optimization
# import official.nlp.bert.bert_models
# import official.nlp.bert.configs
# import official.nlp.bert.run_classifier
# import official.nlp.bert.tokenization
# import official.nlp.data.classifier_data_lib
# import official.nlp.modeling.losses
# import official.nlp.modeling.models
# import official.nlp.modeling.networks

from tqdm import tqdm

In [2]:
! pip install fuzzywuzzy

In [3]:
from fuzzywuzzy import fuzz

/usr/local/lib/python3.7/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


## Install the Kaggle library

In [4]:
!pip install kaggle

## Download API Credentials 

[How to Load Kaggle Datasets Directly into Google Colab?](https://www.analyticsvidhya.com/blog/2021/06/how-to-load-kaggle-datasets-directly-into-google-colab/)

In [5]:
!wget -O kaggle.json https://raw.githubusercontent.com/ShenggKai/Kaggle_TempoRun/master/API_token/kaggle_new.json?token=GHSAT0AAAAAABSGGAGR5354NMW4QPHNTHSMYR2RMMQ

--2022-03-20 14:05:48--  https://raw.githubusercontent.com/ShenggKai/Kaggle_TempoRun/master/API_token/kaggle_new.json?token=GHSAT0AAAAAABSGGAGR5354NMW4QPHNTHSMYR2RMMQ
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 65 [text/plain]
Saving to: ‘kaggle.json’

kaggle.json         100%[===================>]      65  --.-KB/s    in 0s      

2022-03-20 14:05:48 (4.73 MB/s) - ‘kaggle.json’ saved [65/65]



## Setup API credentials 

In [6]:
print("Hien tai ta dang o thu muc:")
!pwd

Hien tai ta dang o thu muc:
/content


In [7]:
ls

kaggle.json  sample_data/


Make a directory named “.kaggle”

In [8]:
! mkdir ~/.kaggle

Copy the “kaggle.json” into this new directory



In [9]:
! cp kaggle.json ~/.kaggle/

In [10]:
!chmod 600 /root/.kaggle/kaggle.json

# Download dataset

In [11]:
# Kiem tra version
!kaggle --version

Kaggle API 1.5.12


In [12]:
!kaggle competitions download -c temporun-retrieval-of-song-lyrics

  0% 0.00/1.35M [00:00<?, ?B/s]
100% 1.35M/1.35M [00:00<00:00, 157MB/s]


Unzip data file

In [13]:
!unzip /content/temporun-retrieval-of-song-lyrics.zip

Archive:  /content/temporun-retrieval-of-song-lyrics.zip
  inflating: 2335_Songs.json         
  inflating: 500_queries.json        
  inflating: sample_submission.csv   


# Preliminaries & Corpus Creation

In [14]:
df = pd.read_json (r'/content/2335_Songs.json')
# df.to_csv (r'/content/sample_data/data.csv', index = None)

In [15]:
df

,author,title,lyrics,song_id
0,Trần Bình,Tình Yêu Thuỷ Chung,"Bài hát: Tình Yêu Thuỷ Chung - Trần Bình, Việt...",0
1,Trần Bình,Lá Thư Của Cha,Bài hát: Lá Thư Của Cha - Trần Bình\nTừ ngày c...,1
2,Đông Nguyễn,Đoạn Buồn Biết Mấy,Đoạn buồn cho tôi biết mấy\nNgười tình rời xa ...,2
3,Cao Hoàng,Chiều Trên Đòi Canve,Can Vê chiều nao lặng lẽ\nThân gầy gục xuống đ...,3
4,Đỗ Minh Hiếu,Mùa Xuân Tình Yêu,Bài hát: Mùa Xuân Tình Yêu - Đỗ Minh Hiếu\nEm ...,4
...,...,...,...,...
2330,ChuChu,Tết Không Cô Vy,"Anh ơi, Xuân đến rồi\nMau mau về nhà thôi\nCô ...",3195
2331,Ng.Huy,Độc Thân Thì Sao,Hook: \nĐộc thân thì nó khác ế không có bồ thì...,3196
2332,Jena,Toxic Love,You are my toxic love\nToxic\nOk babe hãy nói ...,3197
2333,Luân Phan,Anh Đau Lòng Vậy Thôi,"Cố an ủi lòng, ngã đâu đứng lên ở đó \nthật kh...",3198


In [16]:
CORPUS_SONG_JSON = "2335_Songs.json"
QUERIES_JSON = "500_queries.json"

In [17]:
with open(CORPUS_SONG_JSON, 'r', encoding='utf-8') as fo:
  corpus_data = json.load(fo)

with open(QUERIES_JSON, 'r', encoding='utf-8') as fo:
  queries_test = json.load(fo)

# View data

In [18]:
type(corpus_data)

list

In [19]:
print("So hang: ", len(corpus_data))
print("So cot: ", len(corpus_data[0]))

So hang:  2335
So cot:  4


In [20]:
print(corpus_data[652])

{'author': 'Bảo Nguyên', 'title': 'Cánh Thư Mùa Hạ', 'lyrics': 'Hỏi rằng mùa hạ năm nay, \nphương ấy có còn nhiều mây bay?\nHoa nắng lung linh phượng rơi ngủ mơ trên vai em ngọc ngà\nKể từ ngày chia tay đôi nơi,\nBạn bè còn mấy đứa nhắc tới\nEm gái yêu ơi, có còn nhặt hoa hay đã giã từ cuộc chơi?\nNơi xa xôi đóng quân trên vùng sỏi đá\nSuốt năm bốn mùa lửa hạ\nMong một ngày trời nở đầy hoa\nVề trường xưa ngắm lại màu phượng thân yêu\nMuôn sắc hoa dù mỹ miều\nCũng thua màu phượng buồn thật nhiều.\nMấy mùa lửa hạ lại trôi qua,\nThân lính xa nhà còn bôn ba\nNhung nhớ thiết tha phượng rơi ngủ mơ trên vai em ngọc ngà\nVài lời gửi cho cô em tôi\nKỉ niệm ngày trước có nhớ tới?\nXin gửi cho tôi cánh phượng màu môi, cho tôi sống lại ngày vui.\n \n', 'song_id': '0940'}


In [21]:
type(queries_test)

list

In [22]:
print("So hang: ", len(queries_test))
print("So cot: ", len(queries_test[0]))

So hang:  500
So cot:  2


In [23]:
print(queries_test[99])

{'query_id': '099', 'query': 'TA chẳng Còn ChI ĐỂ mà LƯU luYếN Nhau. '}


In [24]:
df.columns

Index(['author', 'title', 'lyrics', 'song_id'], dtype='object')

In [25]:
corpus_data

[{'author': 'Trần Bình',
  'lyrics': 'Bài hát: Tình Yêu Thuỷ Chung - Trần Bình, Việt Linh\nCó phải là duyên khi anh gặp em\nTừ kiếp trước, thề ước chung đôi\nCùng hân hoan\nNhìn nhau đắm đuối\nLòng quyến luyến,\nChuyện mãi không thôi\nEm từ nơi đâu đã đến nơi này\nAnh hân hoan\nTrao Em vòng tay\nVà đôi ta,\nSống chung hạnh phúc\nCùng xây đắp,\nCho mối duyên này\nEm ơi Em ơi, dù phong ba\nDù bão tố, dù nắng cháy\nAnh vẫn yêu Em, mãi mãi không thôi\nAnh ơi Anh ơi, dù gian nan\nDù khốn khó Ta vẫn yêu nhau,\nĐến cuối cuộc đời\nTa vẫn bên nhau qua bao buồn vui\nTa yêu nhau, tình yêu đắm say\nTình yêu ta đẹp, tình yêu thắm thiết\nHạnh phúc trăm năm\nTình yêu thuỷ chung\n \n',
  'song_id': '0000',
  'title': 'Tình Yêu Thuỷ Chung'},
 {'author': 'Trần Bình',
  'lyrics': 'Bài hát: Lá Thư Của Cha - Trần Bình\nTừ ngày con ra đi\nCha lòng buồn thương nhớ\nCô đơn và sầu muộn\nBao ngày tháng mong chờ\nTừ ngày con ra đi\nTới thời gian bây giờ\nThấm thoát mười hai năm chẵn\nCha thường đứng ngồi ngẩn ng

In [26]:
queries_test[499]['query']

'2h sáng overdose'

# CLean data

## Liệt kê các kí tự rác

In [27]:
strash_phrases = ['[', ']', 'hook', ':', 'chorus']
strash_sign = ['.', ',', '(', ')', '?', '\n', '*', "!", "@", '$', '%', '^', '&', '-', '\\', "/", 
               '\'','+', '=', '<', '>', 'ð', '\"', '_']


## Xóa các kí tự rác

### song lyrics

In [28]:
for i in range(2335):
    #Xóa chữ rác
    for phrase in strash_phrases:
        while phrase in corpus_data[i]['lyrics']:
            l = r = corpus_data[i]['lyrics'].find(phrase)

            while l != 0 and corpus_data[i]['lyrics'][l] != '\n':
                l -= 1
            if l != 0:
                l += 1
        
            length = len(corpus_data[i]['lyrics'])

            while r < length and corpus_data[i]['lyrics'][r] != '\n':
                r += 1
        
            corpus_data[i]['lyrics'] = (corpus_data[i]['lyrics'][:l] + corpus_data[i]['lyrics'][r + 1:])
        
    # Xóa dấu rác
    for sign in strash_sign:
        while sign in corpus_data[i]['lyrics']:
            corpus_data[i]['lyrics'] = corpus_data[i]['lyrics'].replace(sign, " ")
    
    output_string = []
    space_flag = False

    # Xóa khoảng trống thừa
    for index in range(len(corpus_data[i]['lyrics'])):
    
        if corpus_data[i]['lyrics'][index] != ' ':
            if space_flag == True:
                output_string.append(' ')
                space_flag = False

            output_string.append(corpus_data[i]['lyrics'][index])
        elif corpus_data[i]['lyrics'][index - 1] != ' ':
            space_flag = True

    corpus_data[i]['lyrics'] = (''.join(output_string))

    # Chuyển tất cả chữ hoa thành chữ thường
    corpus_data[i]['lyrics'] = corpus_data[i]['lyrics'].lower()

In [29]:
# xem kết quả
corpus_data[0]['lyrics']

'có phải là duyên khi anh gặp em từ kiếp trước thề ước chung đôi cùng hân hoan nhìn nhau đắm đuối lòng quyến luyến chuyện mãi không thôi em từ nơi đâu đã đến nơi này anh hân hoan trao em vòng tay và đôi ta sống chung hạnh phúc cùng xây đắp cho mối duyên này em ơi em ơi dù phong ba dù bão tố dù nắng cháy anh vẫn yêu em mãi mãi không thôi anh ơi anh ơi dù gian nan dù khốn khó ta vẫn yêu nhau đến cuối cuộc đời ta vẫn bên nhau qua bao buồn vui ta yêu nhau tình yêu đắm say tình yêu ta đẹp tình yêu thắm thiết hạnh phúc trăm năm tình yêu thuỷ chung'

### queries

In [30]:
for i in range(500):
    # Xóa chữ rác
    for phrase in strash_phrases:
        while phrase in queries_test[i]['query']:
            l = r = queries_test[i]['query'].find(phrase)

            while l != 0 and queries_test[i]['query'][l] != '\n':
                l -= 1
            if l != 0:
                l += 1
        
            length = len(queries_test[i]['query'])

            while r < length and queries_test[i]['query'][r] != '\n':
                r += 1
        
            queries_test[i]['query'] = (queries_test[i]['query'][:l] + queries_test[i]['query'][r + 1:])
        
    # Xóa dấu rác   
    for sign in strash_sign:
        while sign in queries_test[i]['query']:
            queries_test[i]['query'] = queries_test[i]['query'].replace(sign, " ")
    
    output_string = []
    space_flag = False

    # Xóa khoảng trống thừa
    for index in range(len(queries_test[i]['query'])):
    
        if queries_test[i]['query'][index] != ' ':
            if space_flag == True:
                output_string.append(' ')
                space_flag = False

            output_string.append(queries_test[i]['query'][index])
        elif queries_test[i]['query'][index - 1] != ' ':
            space_flag = True
    
    queries_test[i]['query'] = (''.join(output_string))
    
    # Chuyển tất cả chữ hoa thành chữ thường
    queries_test[i]['query'] = queries_test[i]['query'].lower()

In [31]:
# xem kết quả
queries_test[416]['query']

'oh yeah van con đang viết cho em lời họ nói a khum nghe'

## Sửa lại những teencode

### Tải data từ github

In [32]:

!wget -O teencode.csv https://raw.githubusercontent.com/ShenggKai/Kaggle_TempoRun/master/Teencode/teencode.csv

--2022-03-20 14:05:53--  https://raw.githubusercontent.com/ShenggKai/Kaggle_TempoRun/master/Teencode/teencode.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5888 (5.8K) [text/plain]
Saving to: ‘teencode.csv’

teencode.csv        100%[===================>]   5.75K  --.-KB/s    in 0s      

2022-03-20 14:05:53 (75.0 MB/s) - ‘teencode.csv’ saved [5888/5888]



In [33]:
TEENCODE = "/content/teencode.csv"
teencode_list = []
with open(TEENCODE, 'r', encoding='utf-8') as csvfile:
  teencode = csv.reader(csvfile, delimiter=';', quotechar='|')
  for row in teencode:
      teencode_list.append(row)

In [34]:
teencode_list

[['acc', 'account'],
 ['ace', 'anh chị em'],
 ['ad', 'admin'],
 ['ae', 'anh em'],
 ['ah', 'à'],
 ['àh', 'à'],
 ['ạh', 'ạ'],
 ['àk', 'à'],
 ['ák', 'á'],
 ['ạk', 'ạ'],
 ['atsm', 'ảo tưởng sức mạnh'],
 ['atrai', 'anh trai'],
 ['aye', 'anh yêu em'],
 ['baoh', 'bao giờ'],
 ['bằg', 'bằng'],
 ['bbè', 'bạn bè'],
 ['bgái', 'bạn gái'],
 ['biet', 'biết'],
 ['biêt', 'biết'],
 ['bìh', 'bình'],
 ['bik', 'biết'],
 ['bme', 'bố mẹ'],
 ['bn', 'bạn'],
 ['bnhiêu', 'bao nhiêu'],
 ['bnhiu', 'bao nhiêu'],
 ['bóg', 'bóng'],
 ['bsĩ', 'bác sĩ'],
 ['bsvv', 'buổi sáng vui vẻ'],
 ['bth', 'bình thường'],
 ['bthường', 'bình thường'],
 ['btrai', 'bạn trai'],
 ['buôg', 'buông'],
 ['càg', 'càng'],
 ['cbị', 'chuẩn bị'],
 ['cđời', 'cuộc đời'],
 ['ce', 'chị em'],
 ['cgai', 'con gái'],
 ['cgái', 'con gái'],
 ['cj', 'chị'],
 ['ck', 'chồng'],
 ['cka', 'cha'],
 ['clmm', 'cái lồn mẹ mày'],
 ['cmày', 'chúng mày'],
 ['cmm', 'con mẹ mày'],
 ['cmn', 'con mẹ nó'],
 ['cmnd', 'chứng minh nhân dân'],
 ['cmnr', 'con mẹ nó rồi'],
 ['cmt

In [35]:
# chia ra làm hai list cho dễ xử lý
teencode = []
meaning = []
for i in range(len(teencode_list)):
    teencode.append(teencode_list[i][0])
    meaning.append(teencode_list[i][1])

In [36]:
print(teencode)

['acc', 'ace', 'ad', 'ae', 'ah', 'àh', 'ạh', 'àk', 'ák', 'ạk', 'atsm', 'atrai', 'aye', 'baoh', 'bằg', 'bbè', 'bgái', 'biet', 'biêt', 'bìh', 'bik', 'bme', 'bn', 'bnhiêu', 'bnhiu', 'bóg', 'bsĩ', 'bsvv', 'bth', 'bthường', 'btrai', 'buôg', 'càg', 'cbị', 'cđời', 'ce', 'cgai', 'cgái', 'cj', 'ck', 'cka', 'clmm', 'cmày', 'cmm', 'cmn', 'cmnd', 'cmnr', 'cmt', 'csong', 'csống', 'cta', 'ctác', 'cty', 'ctr', 'ctrai', 'ctraj', 'ctrinh', 'ctrình', 'cùg', 'cũg', 'cũng z', 'cv', 'cviec', 'cx', 'chàg', 'chăg', 'chk', 'chs', 'chta', 'chuện', 'chug', 'chúg', 'dàh', 'dc', 'dòg', 'dthương', 'dừg', 'dz', 'dzậy', 'dzề', 'dzìa', 'dzô', 'đag', 'đág', 'đáh', 'đại ka', 'đbh', 'đc', 'đcsvn', 'đhbkhn', 'đhkhtn', 'đhọc', 'đhs', 'địh', 'đii', 'điiiiiii', 'đjên', 'đk', 'đki', 'đkiện', 'đky', 'đóg', 'đôg', 'đồg', 'độg', 'đt', 'đtqg', 'đthoai', 'đthoại', 'đứg', 'đươc', 'đz', 'đzai', 'e', 'eya', 'face', 'fải', 'fây', 'fb', 'fen', 'gắg', 'gđ', 'gđình', 'giốg', 'hahah', 'haj', 'hajzz', 'hđộng', 'hêhê', 'hìh', 'hiha', 'hjhj

In [37]:
print(meaning)

['account', 'anh chị em', 'admin', 'anh em', 'à', 'à', 'ạ', 'à', 'á', 'ạ', 'ảo tưởng sức mạnh', 'anh trai', 'anh yêu em', 'bao giờ', 'bằng', 'bạn bè', 'bạn gái', 'biết', 'biết', 'bình', 'biết', 'bố mẹ', 'bạn', 'bao nhiêu', 'bao nhiêu', 'bóng', 'bác sĩ', 'buổi sáng vui vẻ', 'bình thường', 'bình thường', 'bạn trai', 'buông', 'càng', 'chuẩn bị', 'cuộc đời', 'chị em', 'con gái', 'con gái', 'chị', 'chồng', 'cha', 'cái lồn mẹ mày', 'chúng mày', 'con mẹ mày', 'con mẹ nó', 'chứng minh nhân dân', 'con mẹ nó rồi', 'comment', 'cuộc sống', 'cuộc sống', 'chúng ta', 'công tác', 'công ty', 'chương trình', 'con trai', 'con trai', 'chương trình', 'chương trình', 'cùng', 'cũng', 'cũng vậy', 'công việc', 'công việc', 'cũng', 'chàng', 'chăng', 'chồng', 'chẳng hiểu sao', 'chúng ta', 'chuyện', 'chung', 'chúng', 'dành', 'được', 'dòng', 'dễ thương', 'dừng', 'đẹp trai', 'dậy', 'về', 'về', 'vô', 'đang', 'đáng', 'đánh', 'đại ca', 'đéo bao giờ', 'được', 'đảng cộng sản việt nam', 'đại học bách khoa hà nội', 'đại h

### Sửa teencode

In [38]:
for i in range(500):
    for tc in range(len(teencode)):
        raw_search_string = r"\b" + teencode[tc] + r"\b"

        while re.search(raw_search_string, queries_test[i]['query']) != None:
            queries_test[i]['query'] = queries_test[i]['query'].replace(teencode[tc], '' + meaning[tc])

In [39]:
queries_test[444]['query']

'đay nhung dong nước mắt khẽ vươn tay về thung lũng'

In [40]:
corpus_data[334]['lyrics']

'mùa xuân vừa sang màu thay lá bàn chân ngày đêm mòn sỏi đá dường như từ lâu niềm vui chết từ bao giờ nhin xem thời gian mà ta đã tự thân vượt qua nhiều giông bão giờ đây mình tôi tìm những thứ còn xa vời nằm nghe tiếng mưa rơi thả hồn trôi gió buông lơi tháng ngày tôi lớn lên cùng âm nhạc tìm đâu áng mây trời giữa hàng cây lá xác xơ lúc buồn thì mấy ai hay ở bên xin cho phút giây ngủ yên dài thêm xin mang nỗi đau hòa trong màn đêm hay tôi đã sai vì mãi chạy theo giấc mơ mịt mù xin thôi nhớ mong về những chuyện xưa ai đưa đón tôi vào những ngày mưa xin cho hơi ấm vỗ về ngắm trăng nơi riêng mình xin cho phút giây ngủ yên dài thêm xin mang nỗi đau hòa trong màn đêm xin cho hơi ấm vỗ về ngắm trăng nơi riêng mình xin một lần'

## Xóa dấu cho tiếng việt

Vì thao tác thêm dấu vào dataset tiếng Việt rất phức tạp, nên để đơn giản hơn ta bỏ dấu đồng thời ở `query` và `song_lyric`

In [41]:
# patterns = {
#     '[àáảãạăắằẵặẳâầấậẫẩ]': 'a',
#     '[đ]': 'd',
#     '[èéẻẽẹêềếểễệ]': 'e',
#     '[ìíỉĩị]': 'i',
#     '[òóỏõọôồốổỗộơờớởỡợ]': 'o',
#     '[ùúủũụưừứửữự]': 'u',
#     '[ỳýỷỹỵ]': 'y'
# }

# def convert(text):
#     """
#     Convert from 'Tieng Viet co dau' thanh 'Tieng Viet khong dau'
#     text: input string to be converted
#     Return: string converted
#     """
#     output = text
#     for regex, replace in patterns.items():
#         output = re.sub(regex, replace, output)
#         # deal with upper case
#         output = re.sub(regex.upper(), replace.upper(), output)
#     return output


In [42]:
# for i in range(2335):
#     corpus_data[i]['lyrics'] = convert(corpus_data[i]['lyrics'])

# for i in range(500):
#     queries_test[i]['query'] = convert(queries_test[i]['query'])

In [43]:
# # kiểm tra kết quả
# corpus_data[65]['lyrics']

In [44]:
# queries_test[416]['query']

# Demo

## Biến đổi corpus và query

In [45]:
def read_corpus(corpus_data):
    """
    Function that iteratively reads the saved articles from the corpus directory
    and appends the text to a list.
    
    Parameters
    ----------
    @param corpus_directory: str,
        A valid string specifying the path to the local directory in which the 
        files were saved using make_corpus().
        
    Returns
    -------
    @return corpus, list
        A list containing the text of an article in each element.
    """
    
    # Initialize empty corpus list
    corpus = []
    
    # For each file in the corpus directory, do...
    print("Reading local corpus, please wait...")

    for song in corpus_data:
        if song["lyrics"].split("\n")[0].startswith("Bài hát: "):
            lyric = "\n".join(song["lyrics"].split("\n")[1:])
        else:
            lyric = song["lyrics"]
            
        corpus.append([song["song_id"], lyric])
        
    # Done, return
    print("Done!")
    return corpus

In [46]:
def read_query(query_test):
    query = []
    print("Reading local query, please wait...")

    for qr in query_test:
        temp = qr["query"]

        query.append([qr["query_id"], temp])
    
    print("Done!")
    return query

In [47]:
# Read corpus! 
corpus = read_corpus(corpus_data)

Reading local corpus, please wait...
Done!


In [48]:
queries = read_query(queries_test)

Reading local query, please wait...
Done!


In [49]:
corpus[1]

['0001',
 'từ ngày con ra đi cha lòng buồn thương nhớ cô đơn và sầu muộn bao ngày tháng mong chờ từ ngày con ra đi tới thời gian bây giờ thấm thoát mười hai năm chẵn cha thường đứng ngồi ngẩn ngơ mẹ con không còn nữa đi rồi quên từ giã cha một mình thui thủi lạnh giá và đơn côi nay con ở phương xa đừng quên nơi quê nhà vẫn còn người trông đợi người ấy là cha già cha chờ con trong hấp hối lá thư cuối cùng con ơi thư này mong con nhận được thương con vất vả đơn côi mưa lạnh ngoài trời đang rơi gắng thêm một chút tàn hơi cha đây nhắn lại một lời chúc con hạnh phúc trọn đời trở lại thư này cuối cùng con ơi mẹ con đưa tay vẫy gọi bao năm cha đây trông đợi mẹ đón cha về chung đôi nhớ lời cha dặn con ơi hiến mình giúp đời yên vui dù xa cách biệt phương trời chớ quên tổ quốc giống nòi']

In [50]:
queries[1]

['001', 'chuyện mình giờ kết thúc thật rồi']

## Tiến hành so khớp query với corpus

In [51]:
corpus[0][1]

'có phải là duyên khi anh gặp em từ kiếp trước thề ước chung đôi cùng hân hoan nhìn nhau đắm đuối lòng quyến luyến chuyện mãi không thôi em từ nơi đâu đã đến nơi này anh hân hoan trao em vòng tay và đôi ta sống chung hạnh phúc cùng xây đắp cho mối duyên này em ơi em ơi dù phong ba dù bão tố dù nắng cháy anh vẫn yêu em mãi mãi không thôi anh ơi anh ơi dù gian nan dù khốn khó ta vẫn yêu nhau đến cuối cuộc đời ta vẫn bên nhau qua bao buồn vui ta yêu nhau tình yêu đắm say tình yêu ta đẹp tình yêu thắm thiết hạnh phúc trăm năm tình yêu thuỷ chung'

In [57]:
# Python code to sort the tuples using second element 
# of sublist Inplace way to sort using sort()
def Sort(sub_li):
  
    # reverse = None (Sorts in Ascending order)
    # key is set to sort using second element of 
    # sublist lambda has been used
    sub_li.sort(key = lambda x: x[1], reverse=True)
    return sub_li


def fuzzy_matching(qr, corpus, n_results):
    result = []
    
    top = []
    
    for song in corpus:
        temp = []

        temp.append(song[0])
        temp.append(fuzz.partial_ratio(qr, song[1]))
        

        result.append(temp)
    
    result = Sort(result)

    result = result[:n_results]

    for kq in result:
        top.append(kq[0])

    return top

In [53]:
a = fuzzy_matching(queries[0][1], corpus, 10)

In [54]:
a

['0334',
 '0280',
 '1545',
 '0015',
 '0436',
 '1267',
 '1586',
 '2982',
 '3139',
 '0205']

# Process & Export file submission

In [55]:
with open("submission_winer.csv", "w") as csv_writer:
    fieldnames = ['query_id', 'song_id']
    writer = csv.DictWriter(csv_writer, fieldnames=fieldnames)
    writer.writeheader()

    for ques in tqdm(queries):
        top_results = fuzzy_matching(ques[1], corpus, 10)

        writer.writerow({'query_id': ques[0], "song_id": " ".join(top_results)})

100%|██████████| 500/500 [07:30<00:00,  1.11it/s]


# Other

In [56]:
!kaggle competitions leaderboard temporun-retrieval-of-song-lyrics -s

 teamId  teamName                                        submissionDate       score    
-------  ----------------------------------------------  -------------------  -------  
8298636  StarHào                                         2022-03-16 16:40:52  0.80888  
8308408  Bảo Lương Phạm                                  2022-03-20 13:32:15  0.64362  
8295612  SoulOfWind                                      2022-03-18 08:40:05  0.63000  
8296051  Không Biết Tên                                  2022-03-19 13:33:14  0.60984  
8322762  UITNo2                                          2022-03-19 12:33:05  0.60984  
8295916  Believer                                        2022-03-20 10:48:16  0.60984  
8297820  Chú gà tập code                                 2022-03-20 11:10:09  0.60555  
8299843  TinaMica                                        2022-03-20 13:51:35  0.60555  
8317845  Khánh Nguyên                                    2022-03-20 09:48:38  0.60433  
8314512  No1 UIT                